# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770771369466                   -0.53    9.0         
  2   -2.772138447247       -2.86       -1.31    1.0    138ms
  3   -2.772169997423       -4.50       -2.61    1.0    158ms
  4   -2.772170720890       -6.14       -4.07    2.0    159ms
  5   -2.772170722941       -8.69       -4.56    2.0    176ms
  6   -2.772170723013      -10.14       -5.44    1.0    171ms
  7   -2.772170723015      -11.80       -6.54    1.0    147ms
  8   -2.772170723015      -13.21       -7.00    2.0    185ms
  9   -2.772170723015      -14.45       -7.54    1.0    150ms
 10   -2.772170723015      -15.05       -8.70    1.0    160ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770517194744                   -0.53    8.0         
  2   -2.772050811613       -2.81       -1.29    1.0    167ms
  3   -2.7

1.7735581754443182

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770752838613                   -0.52    9.0         
  2   -2.772058655404       -2.88       -1.32    1.0    198ms
  3   -2.772083115191       -4.61       -2.46    1.0    140ms
  4   -2.772083338422       -6.65       -3.15    1.0    162ms
  5   -2.772083414867       -7.12       -3.79    2.0    192ms
  6   -2.772083417654       -8.55       -4.60    1.0    147ms
  7   -2.772083417808       -9.81       -5.24    2.0    185ms
  8   -2.772083417810      -11.80       -5.75    1.0    149ms
  9   -2.772083417811      -12.29       -6.24    2.0    177ms
 10   -2.772083417811      -13.50       -7.52    1.0    170ms
 11   -2.772083417811      -14.35       -7.91    2.0    170ms
 12   -2.772083417811   +  -13.99       -8.61    1.0    179ms

Polarizability via ForwardDiff:       1.7725349655240454
Polarizability via finite difference: 1.7735581754443182
